In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch
from google.colab import files

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/CS224N

Mounted at /content/drive
/content/drive/MyDrive/CS224N


Dataset Preprocessing

In [ ]:
df = pd.read_csv('train_dataset.csv')
dataset = Dataset.from_pandas(df)

def transform_story(example):
    story_text = example['modified']
    error_position = example['position']

    formatted_text = f'<s>[INST] There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of \'<nl>\' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story_text} [/INST] {error_position} </s>'

    return {'text': formatted_text}

transformed_dataset = dataset.map(transform_story)
for i in range(5):
    print(transformed_dataset[i]['text'])


Finetuning the model

In [6]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

### QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

### TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

### SFT parameters

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("Llama-2-7b-continuity-error-finetuned")

model.eval()

Evaluation on Test Set

In [ ]:
# Similiar to inference for baseline models
from tqdm.auto import tqdm

# Load your test dataset
test_df = pd.read_csv('test_dataset.csv')

# Prepare lists to store true positions, predicted positions, and model responses
y_true = []
y_pred = []
responses = []

# Loop through each item in your test dataset
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['modified']
    true_position = row['position']  # This is the ground truth label for evaluation

    # Construct the prompt without giving away the position
    prompt = f"There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of '<nl>' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story}"

    # Generate the model's output
    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs)  # Adjust max_length if needed
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Store the raw response for later review
    responses.append(response)

    # Extract predicted position from the response, assuming it's a digit sequence
    try:
        predicted_position = int(''.join(filter(str.isdigit, response)))  # Extract digits and convert to integer
        y_pred.append(predicted_position)
    except ValueError:
        y_pred.append(-1)  # Use -1 or another placeholder for non-numeric or missing responses

    y_true.append(true_position)

    # Optionally, print out each response for manual inspection during the evaluation
    print(f"Response: {response}, Predicted Position: {y_pred[-1]}, True Position: {true_position}")

df_response = pd.DataFrame(responses, columns=['response'])
df_preds = pd.DataFrame(y_pred, columns=['preds'])
df_true = pd.DataFrame(y_true, columns=['true'])

df_response.to_csv("test_finetuned_responses.csv", index=False)
df_preds.to_csv("test_finetuned_preds.csv", index=False)
df_true.to_csv("test_finetuned_trues.csv", index=False)

from google.colab import files
files.download("test_finetuned_responses.csv")
files.download("test_finetuned_preds.csv")
files.download("test_finetuned_trues.csv")

Method 2

In [ ]:
# # Load and preprocess the test dataset
# test_df = pd.read_csv('test_dataset.csv')
# test_dataset = Dataset.from_pandas(test_df)

# def transform_story_for_inference(example):
#     story_text = example['modified']
#     formatted_text = f'<s>[INST] There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of \'<nl>\' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story_text} [/INST]</s>'
#     return {'text': formatted_text}

# transformed_test_dataset = test_dataset.map(transform_story_for_inference)

In [ ]:
# # Individual inference, for testing
# # Ignore warnings
# logging.set_verbosity(logging.CRITICAL)

# story_example = transformed_test_dataset[1]['text']
# true_position = transformed_test_dataset[1]['position']

# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"{story_example}")

# predicted_position = result[0]['generated_text'].strip()
# print(f"Predicted position: {predicted_position}, True position: {true_position}")

In [ ]:
# # Predictions for test dataset as a whole
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)  # add max_length, if needed

# def predict_and_compare(dataset):
#     predictions = []
#     references = []

#     for example in dataset:
#         result = pipe(example)
#         predicted_text = result[0]['generated_text'].strip()

#         true_position = example['position']
#         predictions.append(predicted_position)
#         references.append(true_position)

#         print(f"Predicted position: {predicted_position}, True position: {true_position}")

#     # save output
#     predictions_df = pd.DataFrame(predictions, columns=['predicted_position'])
#     references_df = pd.DataFrame(references, columns=['true_position'])

#     predictions_df.to_csv("test_predictions.csv", index=False)
#     references_df.to_csv("test_references.csv", index=False)

#     files.download("test_predictions.csv")
#     files.download("test_references.csv")

# with torch.no_grad():
#   predict_and_compare(transformed_test_dataset)

In [ ]:
# Empty VRAM
del model
# del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, "Llama-2-7b-continuity-error-finetuned")
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"